In [78]:
import json
import pandas as pd
import plotly.express as px
from pathlib import Path

DATA_DIR = Path("../..") / "data"
PEERS_PATH = DATA_DIR / "peers" / "peers.json"
ONLINE_PATH = DATA_DIR / "online_peers" / "online_peers.json"

PEERS_PATH, ONLINE_PATH

(WindowsPath('../../data/peers/peers.json'),
 WindowsPath('../../data/online_peers/online_peers.json'))

In [79]:
def load_json(path):
    with open(path, "r") as f:
        return json.load(f)

peers_raw = load_json(PEERS_PATH)
online_raw = load_json(ONLINE_PATH)

print("Total peers scanned:", len(peers_raw))
print("Total online peers:", len(online_raw))

Total peers scanned: 3828
Total online peers: 1665


In [80]:
df_peers = pd.DataFrame(peers_raw)
df_online = pd.DataFrame(online_raw)

df_peers.head()

,host,ssl,tcp,raw
0,137.184.244.174,NaN,NaN,"[137.184.244.174, 137.184.244.174, [v1.4.3, s5..."
1,194.233.69.180,NaN,50001.0,"[194.233.69.180, electrumx.dev, [v1.4.3, s5000..."
2,37.27.18.174,NaN,NaN,"[37.27.18.174, 37.27.18.174, [v1.4.3, s50002]]"
3,18.221.79.132,NaN,50001.0,"[18.221.79.132, btc5.publicrypto.com, [v1.4.3,..."
4,18.217.9.91,50002.0,50001.0,"[18.217.9.91, btc.byte-share.com, [v1.4.3, s60..."


In [81]:
total = len(peers_raw)
reachable = len(online_raw)
unreachable = total - reachable

metrics = pd.DataFrame({
    "category": ["Reachable", "Unreachable"],
    "count": [reachable, unreachable]
})

metrics

,category,count
0,Reachable,1665
1,Unreachable,2163


In [82]:
fig = px.bar(
    metrics,
    x="category",
    y="count",
    text="count",
    title="Electrum Network Reachability Distribution",
    color="category",
    color_discrete_sequence=["#4CAF50", "#F44336"]
)

fig.update_layout(
    template="plotly_dark",
    height=450,
    xaxis_title="Status",
    yaxis_title="Number of Servers",
)
fig.update_traces(textposition="outside")

fig.show()

In [83]:
fig_pie = px.pie(
    metrics,
    values="count",
    names="category",
    title="Reachability Ratio",
    color="category",
    color_discrete_sequence=["#4CAF50", "#F44336"]
)

fig_pie.update_layout(template="plotly_dark", height=450)
fig_pie.show()

In [84]:
# Código para obter TCP x SSL a partir de df_online

# --- Compute TCP vs SSL availability --------------------------------------

# 1. Cada linha tem "protocol": "tcp" ou "ssl", então basta contar.
proto_stats = (
    df_online["protocol"]
    .value_counts()
    .reset_index()
)

proto_stats.columns = ["protocol", "count"]
proto_stats


,protocol,count
0,tcp,1337
1,ssl,328


In [85]:
fig_proto = px.bar(
    proto_stats,
    x="protocol",
    y="count",
    text="count",
    title="Electrum Server Protocol Availability",
    color="protocol",
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig_proto.update_layout(
    template="plotly_dark",
    height=450,
    xaxis_title="Protocol",
    yaxis_title="Servers",
)

fig_proto.update_traces(textposition="outside")

fig_proto.show()
